In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import numpy as np
from src.preprocess.VoiceBankDEMAND import VoiceBandDEMAND
from src.preprocess.feature_extractor import FeatureExtractor
from src.utils import read_audio, load_yaml
from src.distrib import load_model

# Load the TensorBoard notebook extension.
# %load_ext tensorboard

from tensorflow.python.client import device_lib

# SHOULD PUT model path
model_path = Path(f'./history/221122-1127/data/20221123-183326')
path_conf = os.path.join(model_path, "config.yaml")
args = load_yaml(path_conf)
args.model.path = model_path.as_posix()

Found GPU at: /device:GPU:0Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB




2022-11-29 14:52:30.811249: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-29 14:52:30.811382: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# 1. Load Paramter
device_lib.list_local_devices()
tf.random.set_seed(args.seed)
np.random.seed(args.seed)

model_name = args.model.name
save_path = args.dset.save_path
flag_fft = args.dset.fft
nfft = args.dset.n_fft
hop_length = args.dset.hop_length
center = args.dset.center
num_features = args.model.n_feature
num_segments = args.model.n_segment
normalization = args.dset.normalize
fft_normalization = args.dset.fft_normalize
top_db = args.dset.top_db
train_split = int(args.dset.split*100)
sample_rate = args.dset.sample_rate
win_length = args.dset.win_length

2022-11-29 14:52:31.900350: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-29 14:52:31.900373: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# 3. Build and Load Model
model = load_model(args)

2022-11-29 14:52:31.976699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-29 14:52:31.976728: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 2, 1, 64, 2  0           []                               
                                57)]                                                              
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1, 64, 257)  0           ['input[0][0]']                  
 ingOpLambda)                                                                                     
                                                                                                  
 tf.compat.v1.squeeze (TFOpLamb  (None, 64, 257)     0           ['tf.__operators__.getitem[0][0]'
 da)                                                             ]                            

2022-11-29 14:52:35.346190: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-29 14:52:36.177122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-29 14:52:36.513393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-29 14:52:36.677800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-29 14:52:36.999339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-29 14:52:37.322978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.5156
Optimizer was loaded!


In [4]:

# Load audio file

dataset_path = "/Users/seunghyunoh/workplace/research/NoiseReduction/Tiny-SpeechEnhancement/data/VoiceBankDEMAND/DS_10283_2791"
dataset_voicebank = VoiceBandDEMAND(dataset_path, val_dataset_percent=0.3)
clean_test_filenames, noisy_test_filenames = dataset_voicebank.get_test_filenames()
clean_file = clean_test_filenames[3] 
noisy_file = noisy_test_filenames[3]

clean_file = "/Users/seunghyunoh/workplace/research/NoiseReduction/Tiny-SpeechEnhancement/data/VoiceBankDEMAND/DS_10283_2791/clean_testset_wav/p232_005.wav"
noisy_file = "/Users/seunghyunoh/workplace/research/NoiseReduction/Tiny-SpeechEnhancement/data/VoiceBankDEMAND/DS_10283_2791/noisy_testset_wav/p232_005.wav"

# clean_file = "/home/daniel0413/workplace/project/SpeechEnhancement/example/sample_compare/441c020r/clean.wav"
# noisy_file = "/home/daniel0413/workplace/project/SpeechEnhancement/example/sample_compare/441c020r/dirty.wav"

print("Clean: ", clean_file)
print("Noisy: ", noisy_file)

clean_audio, sr = read_audio(clean_file, sample_rate)
noisy_audio, sr = read_audio(noisy_file, sample_rate)

mean_noisy = np.mean(noisy_audio)
std_noisy = np.std(noisy_audio)
noisy_audio_norm = (noisy_audio - mean_noisy) / std_noisy

noisy_audio_feature_extractor = FeatureExtractor(noisy_audio_norm, windowLength=win_length, hop_length=hop_length, sample_rate=sr)
noisy_stft_features = noisy_audio_feature_extractor.get_stft_spectrogram(center)

noisy_stft_features /= nfft

# Paper: Besides, spectral phase was not used in the training phase.
# At reconstruction, noisy spectral phase was used instead to
# perform in- verse STFT and recover human speech.

noisy_phase = np.angle(noisy_stft_features)
noisy_amplitude = np.abs(noisy_stft_features)


# For metrics
mean_clean = np.mean(clean_audio)
std_clean = np.std(clean_audio)
clean_audio_norm = (clean_audio - mean_clean) / std_clean
clean_audio_feature_extractor = FeatureExtractor(clean_audio_norm, windowLength=win_length, hop_length=hop_length, sample_rate=sr)
clean_stft_features = clean_audio_feature_extractor.get_stft_spectrogram(center)
clean_stft_features /= nfft
clean_phase = np.angle(clean_stft_features)
clean_amplitude = np.abs(clean_stft_features)

print("Min:", np.min(clean_amplitude),"Max:",np.max(clean_amplitude))




File example:
Clean:  /Users/seunghyunoh/workplace/research/NoiseReduction/Tiny-SpeechEnhancement/data/VoiceBankDEMAND/DS_10283_2791/clean_testset_wav/p232_001.wav The number:  824
Noisy:  /Users/seunghyunoh/workplace/research/NoiseReduction/Tiny-SpeechEnhancement/data/VoiceBankDEMAND/DS_10283_2791/noisy_testset_wav/p232_001.wav The number:  824
# of Noise testing files: 824
Clean:  /Users/seunghyunoh/workplace/research/NoiseReduction/Tiny-SpeechEnhancement/data/VoiceBankDEMAND/DS_10283_2791/clean_testset_wav/p232_005.wav
Noisy:  /Users/seunghyunoh/workplace/research/NoiseReduction/Tiny-SpeechEnhancement/data/VoiceBankDEMAND/DS_10283_2791/noisy_testset_wav/p232_005.wav
Min: 1.3051452280032905e-11 Max: 0.6614166143827909


OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [5]:
def _prepare_input_features(stft_features, numSegments, numFeatures):
    stftSegments = np.zeros((numFeatures, numSegments, stft_features.shape[1]))
    for index in range(stft_features.shape[1]-numSegments+1):
        stftSegments[..., index] = stft_features[..., index:index + numSegments]
    return stftSegments

def _prepare_input_features_zero_filled(stft_features, numSegments, numFeatures):
    noisySTFT = np.concatenate([np.zeros_like(stft_features[:, 0:numSegments - 1]), stft_features], axis=1)
    stftSegments = np.zeros((numFeatures, numSegments, noisySTFT.shape[1] - numSegments + 1))
    for index in range(noisySTFT.shape[1] - numSegments + 1):
        stftSegments[..., index] = noisySTFT[..., index:index + numSegments]
    return stftSegments


def revert_features_to_audio(features, phase, mean=None, std=None):
    features = np.squeeze(features)

    # features = librosa.db_to_power(features)
    features = features * np.exp(1j * phase)  # that fixes the abs() ope previously done

    features = np.transpose(features, (1, 0))
    features *= nfft
    estimated_audio = noisy_audio_feature_extractor.get_audio_from_stft_spectrogram(features, center)
    
    # scale the outpus back to the original range
    if mean and std:
        estimated_audio = std * estimated_audio + mean
    
    return estimated_audio


noisy_ampltidue_input = _prepare_input_features_zero_filled(noisy_amplitude, num_segments, num_features)
noisy_ampltidue_input = np.transpose(noisy_ampltidue_input, (2, 1, 0)).astype(np.float32)

noisy_phase_input = _prepare_input_features_zero_filled(noisy_phase, num_segments, num_features)
noisy_phase_input = np.transpose(noisy_phase_input, (2, 1, 0)).astype(np.float32)

noisy_input = np.stack([noisy_ampltidue_input, noisy_phase_input], axis=1)
noisy_input = np.expand_dims(noisy_input, axis=2)
estimation_amp_phase = model.predict(noisy_input)

estimation_amp_phase = estimation_amp_phase[..., -1, :]
estimation_amp_phase = np.squeeze(estimation_amp_phase)

if model_name == 'lstm':
    estimation = revert_features_to_audio(estimation_amp_phase[:, 0, :], estimation_amp_phase[:, 1, :], mean_noisy, std_noisy)

print("Min:", np.min(estimation),"Max:",np.max(estimation))
# ipd.Audio(data=estimation, rate=fs) # load a local WAV file

2022-11-29 14:52:38.507577: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-29 14:52:38.661379: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 1/13 [=>............................] - ETA: 8s

2022-11-29 14:52:38.832533: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 1s 50ms/step
Min: -0.5340567 Max: 0.42722604


In [17]:

noisy_bypass = noisy_input[..., -1, :]
noisy_bypass = np.squeeze(noisy_bypass)
noisy_bypass = revert_features_to_audio(noisy_bypass[:, 0, :], noisy_bypass[:, 1, :], mean_noisy, std_noisy)

clean_ampltidue_input = _prepare_input_features_zero_filled(clean_amplitude, num_segments, num_features)
clean_ampltidue_input = np.transpose(clean_ampltidue_input, (2, 1, 0)).astype(np.float32)

clean_phase_input = _prepare_input_features_zero_filled(clean_phase, num_segments, num_features)
clean_phase_input = np.transpose(clean_phase_input, (2, 1, 0)).astype(np.float32)

clean_input = np.stack([clean_ampltidue_input, clean_phase_input], axis=1)
clean_input = np.expand_dims(clean_input, axis=2)

clean_bypass = clean_input[..., -1, :]
clean_bypass = np.squeeze(clean_bypass)
clean_bypass = revert_features_to_audio(clean_bypass[:, 0, :], clean_bypass[:, 1, :], mean_clean, std_clean)

from src.metrics import SI_SDR

metric = SI_SDR

estimation_metric = estimation

new_shape = list(clean_bypass.shape)
nsegment_metric = int(new_shape[-1]//sample_rate)
new_shape += [0]
new_shape[-2:] = nsegment_metric, sample_rate

clean_bypass = clean_bypass[:nsegment_metric*sample_rate]
clean_bypass = np.reshape(clean_bypass, new_shape)
noisy_bypass = noisy_bypass[:nsegment_metric*sample_rate]
noisy_bypass = np.reshape(noisy_bypass, new_shape)
estimation_metric = estimation[:nsegment_metric*sample_rate]
estimation_metric = np.reshape(estimation_metric, new_shape)

for clean_seg, noisy_seg, est_seg in zip(clean_bypass, noisy_bypass, estimation_metric):
    SISDR_prev = metric(clean_seg, noisy_seg, sample_rate)
    SISDR_after = metric(clean_seg, est_seg, sample_rate)
    print(SISDR_prev, "---->", SISDR_after)

SISDR_prev = metric(clean_bypass, noisy_bypass, sample_rate)
SISDR_after = metric(clean_bypass, estimation_metric, sample_rate)
print(SISDR_prev, "---->", SISDR_after)


-5.070292353630066 ----> 2.924632728099823
0.9577148407697678 ----> 5.680829882621765
-0.144052030518651 ----> 7.788050174713135
3.0900216102600098 ----> 5.8359068632125854
11.649209260940552 ----> 7.882385849952698
5.249779224395752 ----> 6.3077312707901
5.746272802352905 ----> 6.354302763938904


In [ ]:
# plot
f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharey=True)

ax1.plot(clean_audio)
ax1.set_title("Clean Audio")

ax2.plot(noisy_audio)
ax2.set_title("Noisy Audio")

ax3.plot(estimation)
ax3.set_title("Denoised Audio")


In [ ]:
import IPython.display as ipd

In [ ]:
ipd.Audio(data=clean_audio, rate=sample_rate) # load a local WAV file

In [ ]:
ipd.Audio(data=noisy_audio, rate=sample_rate) # load a local WAV file

In [ ]:
ipd.Audio(data=estimation, rate=sample_rate) # load a local WAV file

In [ ]:
import numpy as np
import librosa
import librosa.display

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, sharey=True)

def show_stft(y, _fig, _ax):
    D = librosa.stft(y)  # STFT of y
    S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)
    img = librosa.display.specshow(S_db, ax=_ax)
    _fig.colorbar(img, ax=_ax)

show_stft(estimation, fig, ax0)
show_stft(clean_audio, fig, ax1)
show_stft(noisy_audio, fig, ax2)

In [ ]:
# bose_file = "/home/daniel0413/workplace/project/SpeechEnhancement/example/sample_compare/441c020r/proc.wav"
# boseAudio, sr = read_audio(bose_file, sample_rate)
# ipd.Audio(data=boseAudio, rate=sample_rate)